In [ ]:
# Environment & determinism (benchmark)
import os, sys, json, random
SEED = 123
random.seed(SEED)
print(json.dumps({'seed': SEED, 'ci': os.getenv('CI','0')=='1'}, indent=2))

# Restoria Benchmark

Run quick benchmarks comparing Restoria backends, modes (eager/torch.compile/ORT), and metrics.

## Optional extras (Colab only)
Install optional ORT CPU and metrics (LPIPS/DISTS). ArcFace is heavy and skipped by default. Safe to skip.

In [ ]:
import os, sys, subprocess
IN_COLAB = "COLAB_GPU" in os.environ or "google.colab" in sys.modules
if IN_COLAB and not os.environ.get('NB_CI_SMOKE'):
    def pip_install(pkgs):
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)
        except Exception as e:
            print('[WARN] pip failed:', e)
    extras = [
        'onnxruntime>=1.17,<2',
        'lpips>=0.1.4',
        'git+https://github.com/richzhang/PerceptualSimilarity.git@master#egg=lpips-extra',
        "torchdists>=0.1.0; python_version>='3.9'",
    ]
    print('[INFO] Installing optional extras…')
    pip_install(extras)
else:
    print('[INFO] Extras install skipped (not in Colab or CI).')

In [ ]:
# Quickstart (dry-run) benchmark
import os, sys
from pathlib import Path
if 'restoria' not in sys.modules:
    sys.path.insert(0, str(Path.cwd() / 'src'))
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png' if os.path.exists('assets/gfpgan_logo.png') else 'tmp/blank.png'
out_dir = 'results/dry_demo/benchmark'
rc = restoria_main(['run','--input', img,'--output', out_dir,'--dry-run','--device','auto'])
print('rc:', rc)

### Real run (optional)

Collect quick metrics across modes and backends (keep inputs small in Colab):

```python
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png'
for be in ['gfpgan','gfpgan-ort']:
    for compile_flag in [False, True]:
        out_dir = f'results/real_demo/bench_{be}_compile_{int(compile_flag)}'
        args = ['run','--input', img,'--output', out_dir,'--device','auto','--metrics','fast']
        if compile_flag:
            args.append('--compile')
        restoria_main(args)
```

## Plan and outputs summary
Quickly inspect the chosen device, backend, and a few metrics from your run.

In [ ]:
import json, os

def summarize_out(out_dir):
    man = os.path.join(out_dir, 'manifest.json')
    met = os.path.join(out_dir, 'metrics.json')
    if os.path.exists(man):
        m = json.load(open(man))
        env = m.get('env', {})
        plan = m.get('plan', {})
        print('device:', env.get('resolved_device') or env.get('device'))
        if plan:
            print('backend:', plan.get('backend'), '| reason:', plan.get('reason'))
    else:
        print('manifest.json missing')
    if os.path.exists(met):
        data = json.load(open(met))
        rows = data.get('records') or []
        print('metrics records:', len(rows))
        if rows:
            m0 = rows[0].get('metrics') or {}
            for k in ['arcface_cosine','lpips','dists']:
                if k in m0:
                    print(k, '=', m0[k])
    else:
        print('metrics.json missing')

out_dir = globals().get('out_dir', 'results/dry_demo/bench_demo')
try:
    summarize_out(out_dir)
except Exception as e:
    print('[WARN] summary failed:', e)